In [142]:
from dotenv import load_dotenv
from IPython.display import Markdown
from openai import OpenAI
from pypdf import PdfReader
import os
import gradio as gr


In [136]:
load_dotenv(override=True)

True

In [137]:
openai = os.getenv("OPENAI_API_KEY")
groqai = os.getenv("groq_api_key")

if openai:
    print(f"Api key for openai is found and starts with: {openai[:8]}")
else:
    print("key noy found.Check guide")
if groqai:
    print(f"APi key for groqai is found and starts with: {groqai[:8]}")
else:
    print("groq api key not found")

Api key for openai is found and starts with: sk-proj-
APi key for groqai is found and starts with: gsk_Vopn


In [140]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [146]:
reader = PdfReader("me/Profile.pdf")

linkedin= ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text



In [ ]:
name = "Oluwatosin"

In [147]:
system_prompt = f"You are asking question about {name} website,\
particularly questions related to {name} career , background, skills and experience.\
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [149]:
import openai

def chat(message, history):

    message = [{"role":"system","content":system_prompt}] + history + [{"role":"user","content":message}]

    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = message
    )
    return response.choices[0].message.content

In [152]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [ ]:
#Time to evaluate the model - Aim is to build a Multi-LLM pipeline
#We will use the groqapi to evaluate the openai model

#First import a pydantc library and a basemodel class

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [154]:
#create an evaluator variable

evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [155]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the user and the agent:\n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the user:\n\n{message}\n\n"
    user_prompt +=f"Here's the latest response from the agent:\n\n{reply}\n\n"
    return user_prompt

In [156]:
#import and set enviroment for the groqai

groqapi = OpenAI(api_key=groqai,
                base_url="https://api.groq.com/openai/v1"
                )

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role":"system","content":evaluator_system_prompt}] + [{"role":"user","content":evaluator_user_prompt(reply,message,history)}]
    response = groqapi.chat.completions.create(
        model="llama3-8b-8192",
        messages = messages,
        #response_format=Evaluation
    )

    raw_content = response.choices[0].message.content

    try:
        # If response is a JSON string: {"is_acceptable": true, "feedback": "..."}
        #using this deprectaed - evaluation = Evaluation.parse_raw(raw_content) - deprecated
        evaluation = Evaluation.model_validate_json(raw_content)
    except:
        # Otherwise, fallback to plain text evaluation if it's not JSON
        evaluation = Evaluation(
            is_acceptable="acceptable" in raw_content.lower(),
            feedback=raw_content
        )

    return evaluation


    #return response.choices[0].message.content

In [180]:
messages = [{"role":"system", "content":"system_prompt"}] + [{"role":"user", "content":"do you hold a patent"}]
response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages
)

reply = response.choices[0].message.content

In [181]:
evaluate(reply,"do you hold a patent?",messages[:1])

Evaluation(is_acceptable=True, feedback='I evaluate the latest response from the agent as ACCEPTABLE.\n\nThe response is well-structured, concise, and directly addresses the user\'s question. The agent acknowledges that they don\'t hold a patent, which is an honest and clear answer. Additionally, the agent proactively offers to provide information on patents, application processes, or discuss patent law if the user has further questions, showing their willingness to engage and be helpful.\n\nFeedback:\nThe response effectively addresses the user\'s query, and the agent\'s tone is professional and engaging. However, to further improve, the agent could consider adding a brief sentence or phrase to emphasize their expertise in the field of Data and AI, such as "As a Data and AI Practitioner, I can provide insights on the patent process in my area of specialization." This would help to reinforce their credibility and expertise while maintaining the response\'s overall length.')

In [182]:
def rerun(reply,message,history,feedback):

    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role":"system", "content":updated_system_prompt}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    response.choices[0].message.content

In [191]:
def chat(message, history):
    if "patent" not in message:
        system = system_prompt + "\n\n Everything in the reply needs to be in pig latin.It is mandatory that you respond and only entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role":"system","content":system}]+ history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages
    )

    reply = response.choices[0].message.content

    evaluation = evaluate(reply,message,history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation = retrying")
        print(evaluation.reply)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [192]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
